In [ ]:
import os

os.environ["HF_HOME"] = "/DATA2/HuggingFace"

from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
import numpy as np


In [ ]:
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True) 
model.to("cuda:0")

# prepare image and text prompt, using the appropriate prompt template

# image_path = "/home/mou/Projects/DeepHallu/data/examples/image.jpeg"
# image = Image.open(image_path)
# question = "What is shown in this image?"

from datasets import load_dataset
llava_bench = load_dataset("lmms-lab/llava-bench-in-the-wild")
sample = llava_bench['train'][0]
image = sample['image']
question = sample['question']
print(question)
# Define a chat history and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image") 
conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": question},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda:0")
# # Explore the inputs
# # image size
# print(f"image size: {image.size}")
# # inputs contains:
# # - input_ids
# # - attention_mask
# # - pixel_values
# # - image_sizes
# for k, v in inputs.items():
#     print(k, v.shape)
# print(f"inputs: {inputs['image_sizes']}")
# # Decode and print the input_ids as text for inspection
# decoded_input = processor.tokenizer.decode(input_ids, skip_special_tokens=False)
# print("Decoded input_ids:\n", decoded_input)
# image_token_id = model.config.image_token_id
# input_ids = inputs["input_ids"][0]
# image_token_indices = (input_ids == image_token_id).nonzero(as_tuple=True)[0].tolist()
# print("Indices of image_token_id in input_ids:", image_token_indices)

In [ ]:

for k, v in inputs.items():
    print(k, v.shape)

In [ ]:
print(f"inputs: {inputs['image_sizes']}")
# print(processor.tokenizer.decode(inputs["input_ids"][0]))
print(prompt)

In [ ]:
# autoregressively complete prompt
output = model.generate(**inputs, max_new_tokens=100)
print(processor.decode(output[0], skip_special_tokens=True))

In [ ]:

for k, v in inputs.items():
    print(k, v.shape)
image_token_id = model.config.image_token_id
input_ids = inputs["input_ids"][0]
image_token_indices = (input_ids == image_token_id).nonzero(as_tuple=True)[0].tolist()
print(f"Number of image tokens: {len(image_token_indices)}")